In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpa

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-05-05 02:29:26--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  6.05MB/s    in 0.2s    

2021-05-05 02:29:26 (6.05 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PGA_data").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://red-team-final.s3.amazonaws.com/reshaped_cleaned_dataset.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("reshaped_cleaned_dataset.csv"), sep=",", header=True, inferSchema=True)
df.show()

+------------------+--------------------+-----------------+-------------------+--------------+-------------------------+---------+-----------+-------------+-----------------+------------+--------------+--------------+---------------+------------+---------------+--------------+----------------------+------------+----------------+-------------------------+-------------------------------+-----------+
|SG_off_tee_AVERAGE|SG_tee_green_AVERAGE|driving_dist_AVG.|pct_ydg_tee_AVG (%)|driving_320+_%|SG_approach_green_AVERAGE|GIR_pct_%|GOFIR_pct_%|GIR_pct_fwy_%|GIR_pct_not_fwy_%|fwy_prox_AVG|rough_prox_AVG|SG_ATG_AVERAGE|pct_sand_save_%|scrambling_%|SG_putt_AVERAGE|pct_one_putt_%|pct_three_putt_avoid_%|putt_avg_AVG|putts_per_rd_AVG|bogey_avoid_% MAKES BOGEY|btb_ratio_BIRDIE TO BOGEY RATIO|money_MONEY|
+------------------+--------------------+-----------------+-------------------+--------------+-------------------------+---------+-----------+-------------+-----------------+------------+-----------